In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [3]:
def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file._append(temp_df)
    file.to_csv(filename + '.csv')

In [7]:
#!pip install geopy


In [9]:
import geopy


In [11]:
from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [13]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results

In [21]:
prices = pd.read_csv('ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

In [23]:
prices = prices[['block', 'street_name']]
prices['address'] = prices['block'] + ' ' + prices['street_name']
all_address = list(prices['address'])
unique_address = list(set(all_address))

print('Unique addresses:', len(unique_address))

Unique addresses: 9610


In [25]:
unique_address[:10]

['469 ADMIRALTY DR',
 '168B PUNGGOL EAST',
 '577 ANG MO KIO AVE 10',
 '50 CIRCUIT RD',
 '250 BISHAN ST 22',
 '819 WOODLANDS ST 82',
 '606 CHOA CHU KANG ST 62',
 '932 YISHUN CTRL 1',
 '65 TELOK BLANGAH DR',
 '137 YISHUN RING RD']

In [ ]:
#find_postal(unique_address, 'Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/flat_coordinates2')

In [ ]:
# find_postal(unique_address, 'Data/flat_coordinates')

In [45]:
flat_coord = pd.read_csv('hdb_coordinate.csv')
flat_coord = flat_coord[['Address','Latitude','Longitude']]
flat_coord.head(10)

,Address,Latitude,Longitude
0,1 BEACH RD,1.303671,103.864479
1,1 BEDOK STH AVE 1,1.320852,103.933721
2,1 CANTONMENT RD,1.275499,103.841398
3,1 CHAI CHEE RD,1.327969,103.922716
4,1 CHANGI VILLAGE RD,1.388547,103.987804
5,1 DELTA AVE,1.292075,103.828584
6,1 DOVER RD,1.302530,103.783272
7,1 EUNOS CRES,1.320658,103.902463
8,1 EVERTON PK,1.275499,103.841398
9,1 GEYLANG SERAI,1.316679,103.898103


# Supermarket

In [49]:
supermarket = pd.read_csv('list-of-supermarket-licences.csv')
supermarket.head(10)

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913
5,CE14L50J000,NTUC Fairprice Co-operative Ltd,THE SELETAR MALL,33,B2,24,SENGKANG WEST AVENUE,797653
6,CE12H94B000,PRIME SUPERMARKET (1996) PTE LTD,MUTLI STOREY CAR PARK,542B,2,1,SERANGOON NORTH AVENUE 3,552542
7,CE16913J000,SHENG SIONG SUPERMARKET PTE LTD,FERNVALE RIVERGROVE,473,1,6,FERNVALE STREET,790473
8,CE15042P000,D-RON INVESTMENTS PTE. LTD.,PPT LODGE 1B,2,1,38,SELETAR NORTH LINK,797601
9,CE15023A000,GW TRADER PTE. LTD.,PPT LODGE 1A,8,G1,3,SELETAR NORTH LINK,797455


In [51]:
supermerket_address = list(supermarket['postal_code'])
unique_supermarket_address = list(set(supermerket_address))

print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [53]:
supermarket_coord = pd.read_csv('supermarket_coordinates.csv')
supermarket_coord.drop_duplicates(subset=['address'], inplace=True)
supermarket_coord = supermarket_coord[['SEARCHVAL','LATITUDE','LONGITUDE']]
supermarket_coord.head(10)

,SEARCHVAL,LATITUDE,LONGITUDE
0,GOLDEN VILLAGE (GV KATONG),1.305225,103.905102
4,MULTI STOREY CAR PARK,1.273388,103.808981
8,11 RIVERVALE CRESCENT SINGAPORE 545082,1.392504,103.904484
14,CORALINUS,1.403719,103.906138
16,WESTLITE JUNIPER,1.406303,103.760615
17,TAMAN JURONG SHOPPING CENTRE,1.334845,103.720462
26,376 BUKIT BATOK STREET 31 SINGAPORE 650376,1.358317,103.749580
27,SERANGOON GARDEN ESTATE,1.364281,103.865134
29,SINGAPORE VETERINARY ANIMAL CLINIC,1.445813,103.798219
36,OCBC NEX - FAIRPRICE,1.350438,103.871705


In [55]:
nearest_supermarket = find_nearest(flat_coord, supermarket_coord)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T
flat_supermarket = flat_supermarket.rename(columns={0: 'flat', 1: 'supermarket', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop(['index','supermarket'], axis=1)
flat_supermarket.head()

,flat,supermarket_dist,num_supermarket_2km
0,1 BEACH RD,0.125944,24
1,1 BEDOK STH AVE 1,0.115509,20
2,1 CANTONMENT RD,0.196809,18
3,1 CHAI CHEE RD,0.399689,20
4,1 CHANGI VILLAGE RD,3.325376,0


In [57]:
flat_supermarket.to_csv('flat_supermarket.csv', index=False)

# School

In [61]:
school = pd.read_csv('school_names.csv', encoding='cp1252')
school.head(10)

,school,area
0,Admiralty Primary School,Woodlands
1,Ahmad Ibrahim Primary School,Yishun
2,Ai Tong School,Bishan
3,Alexandra Primary School,Bukit Merah
4,Anchor Green Primary School,Sengkang
5,Anderson Primary School,Ang Mo Kio
6,Anglo-Chinese School (Junior),Central
7,Anglo-Chinese School (Primary),Bukit Timah
8,Angsana Primary School,Tampines
9,Ang Mo Kio Primary School,Ang Mo Kio


In [63]:
school_name = list(school['school'])
unique_school_name = list(set(school_name))

print('Unique addresses:', len(unique_school_name))

Unique addresses: 191


In [69]:
school_coord = pd.read_csv('school_coordinates.csv')
school_coord = school_coord[['address','LATITUDE','LONGITUDE']]
school_coord.head()

,address,LATITUDE,LONGITUDE
0,Princess Elizabeth Primary School,1.349195,103.741000
1,Woodgrove Primary School,1.433058,103.790390
2,Nanyang Primary School,1.321115,103.806468
3,Nanyang Primary School,1.321246,103.807795
4,Waterway Primary School,1.399154,103.918671


In [71]:
nearest_school = find_nearest(flat_coord, school_coord)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'flat', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head(10)

,flat,school,school_dist,num_school_2km
0,1 BEACH RD,Stamford Primary School,0.789938,2
1,1 BEDOK STH AVE 1,Bedok Green Primary School,0.566936,7
2,1 CANTONMENT RD,Cantonment Primary School,0.164246,5
3,1 CHAI CHEE RD,Damai Primary School,0.832069,8
4,1 CHANGI VILLAGE RD,Pasir Ris Primary School,3.300565,0
5,1 DELTA AVE,Alexandra Primary School,0.523839,7
6,1 DOVER RD,Fairfield Methodist School (Primary),0.330473,3
7,1 EUNOS CRES,Eunos Primary School,0.464139,7
8,1 EVERTON PK,Cantonment Primary School,0.164246,5
9,1 GEYLANG SERAI,Haig Girls' School,0.742367,8


In [75]:
flat_school.to_csv('flat_school.csv', index=False)

# Hawkers and Markets

In [79]:
hawker = pd.read_csv('list-of-government-markets-hawker-centres.csv')
hawker.head(10)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0
5,Bukit Timah Market,"51, Upper Bukit Timah Road, S(588215)",MHC,Government,179,84,95
6,Chomp Chomp Food Centre,"20, Kensington Park Road, S(557269)",HC,Government,36,36,0
7,Commonwealth Crescent Market,"31, Commonwealth Crescent, S(149644)",MHC,Government,102,39,63
8,Dunman Food Centre,"271, Onan Road, S(424768)",HC,Government,30,30,0
9,East Coast Lagoon Food Village,"1220, East Coast Parkway, S(468960)",HC,Government,63,63,0


In [81]:
hawker_name = list(hawker['name_of_centre'])
unique_hawker_name = list(set(hawker_name))

print('Unique addresses:', len(unique_hawker_name))

Unique addresses: 107


In [83]:
hawker_coord = pd.read_csv('hawker_coordinates.csv')
hawker_coord = hawker_coord[['address','LATITUDE','LONGITUDE']]
hawker_coord.head(10)

,address,LATITUDE,LONGITUDE
0,Amoy Street Food Centre,1.279340,103.846653
1,Blks 20/21 Marsiling Lane,1.443444,103.776989
2,Blks 20/21 Marsiling Lane,1.443832,103.777210
3,Geylang Serai Market,1.316725,103.897977
4,Blks 2 & 3 Changi Village Road,1.443444,103.776989
5,Blks 2 & 3 Changi Village Road,1.443832,103.777210
6,Blks 2 & 3 Changi Village Road,1.389649,103.988100
7,Blks 2 & 3 Changi Village Road,1.389177,103.988328
8,East Coast Lagoon Food Village,1.306895,103.935121
9,Blk 112 Jalan Bukit Merah,1.280005,103.826013


In [85]:
nearest_hawker = find_nearest(flat_coord, hawker_coord)
flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
flat_hawker = flat_hawker.rename(columns={0: 'flat', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
flat_hawker.head(10)

,flat,hawker,hawker_dist,num_hawker_2km
0,1 BEACH RD,Golden Mile Food Centre,0.110155,7
1,1 BEDOK STH AVE 1,Blk 16 Bedok South Road,0.198866,8
2,1 CANTONMENT RD,Blk 6 Tanjong Pagar Plaza,0.232288,10
3,1 CHAI CHEE RD,Blk 538 Bedok North Street 3,0.498505,6
4,1 CHANGI VILLAGE RD,Blks 2 & 3 Changi Village Road,0.090824,2
5,1 DELTA AVE,Zion Riverside Food Centre,0.291435,9
6,1 DOVER RD,Blk 20 Ghim Moh Road,1.079151,5
7,1 EUNOS CRES,Blk 4A Eunos Crescent,0.210803,6
8,1 EVERTON PK,Blk 6 Tanjong Pagar Plaza,0.232288,10
9,1 GEYLANG SERAI,Geylang Serai Market,0.014855,10


In [90]:
flat_hawker.to_csv('flat_hawker.csv', index=False)

# Shopping Malls

In [94]:
shop = pd.read_csv('shoppingmall_coordinates_clean.csv', encoding='cp1252')
shop.head(10)

,address,LATITUDE,LONGITUDE
0,Katong Square,1.304853,103.904574
1,PLQ Mall,1.317482,103.892832
2,Capitol Piazza,1.293063,103.851293
3,Bugis Cube,1.298195,103.855655
4,HillV2,1.363551,103.764236
5,100 AM,1.274588,103.843471
6,Midpoint Orchard,1.301784,103.838661
7,Woodlands Mart,1.445648,103.798162
8,Bukit Timah Plaza,1.338632,103.778585
9,Funan,1.291244,103.849985


In [98]:
shop_name = list(shop['address'])
unique_shop_name = list(set(shop_name))

print('Unique addresses:', len(unique_shop_name))

Unique addresses: 159


In [102]:
shop_coord = pd.read_csv('shoppingmall_coordinates_clean.csv')
shop_coord.drop_duplicates(subset=['address'], inplace=True)
shop_coord = shop_coord[['address','LATITUDE','LONGITUDE']]
shop_coord.head(10)

,address,LATITUDE,LONGITUDE
0,Katong Square,1.304853,103.904574
1,PLQ Mall,1.317482,103.892832
2,Capitol Piazza,1.293063,103.851293
3,Bugis Cube,1.298195,103.855655
4,HillV2,1.363551,103.764236
5,100 AM,1.274588,103.843471
6,Midpoint Orchard,1.301784,103.838661
7,Woodlands Mart,1.445648,103.798162
8,Bukit Timah Plaza,1.338632,103.778585
9,Funan,1.291244,103.849985


In [104]:
nearest_mall = find_nearest(flat_coord, shop_coord)
flat_mall = pd.DataFrame.from_dict(nearest_mall).T
flat_mall = flat_mall.rename(columns={0: 'flat', 1: 'mall', 2: 'mall_dist', 3: 'num_mall_2km'}).reset_index().drop('index', axis=1)
flat_mall.head(10)

,flat,mall,mall_dist,num_mall_2km
0,1 BEACH RD,Aperia,0.672618,18
1,1 BEDOK STH AVE 1,Bedok Point,0.461313,5
2,1 CANTONMENT RD,100 AM,0.251648,10
3,1 CHAI CHEE RD,Djitsun Mall,0.795355,4
4,1 CHANGI VILLAGE RD,Jewel Changi Airport,3.126753,0
5,1 DELTA AVE,Great World City,0.390828,26
6,1 DOVER RD,Rochester Mall,0.657988,4
7,1 EUNOS CRES,Singapore Post Centre (Singpost Centre),0.87899,10
8,1 EVERTON PK,100 AM,0.251648,10
9,1 GEYLANG SERAI,Singapore Post Centre (Singpost Centre),0.443402,11


In [106]:
flat_mall.to_csv('flat_mall.csv', index=False)

# Parks

In [108]:
park_coord = pd.read_csv('parks_coordinates_clean.csv')
park_coord.reset_index(inplace=True)
park_coord = park_coord[['index','Y','X']]
park_coord.head(10)

,index,Y,X
0,0,1.346175,103.960857
1,1,1.389904,103.978979
2,2,1.409913,103.923131
3,3,1.379237,103.866967
4,4,1.372761,103.829073
5,5,1.320710,103.951754
6,6,1.340659,103.955192
7,7,1.366937,103.881611
8,8,1.390098,103.834277
9,9,1.381099,103.861151


In [110]:
nearest_park = find_nearest(flat_coord, park_coord)
flat_park = pd.DataFrame.from_dict(nearest_park).T
flat_park = flat_park.rename(columns={0: 'flat', 1: 'park', 2: 'park_dist', 3: 'num_park_2km'}).reset_index().drop(['index','park'], axis=1)
flat_park.head(10)

,flat,park_dist,num_park_2km
0,1 BEACH RD,0.21358,14
1,1 BEDOK STH AVE 1,0.507003,22
2,1 CANTONMENT RD,0.621771,14
3,1 CHAI CHEE RD,0.673814,16
4,1 CHANGI VILLAGE RD,0.234404,7
5,1 DELTA AVE,0.386117,11
6,1 DOVER RD,0.661312,14
7,1 EUNOS CRES,0.69627,14
8,1 EVERTON PK,0.621771,14
9,1 GEYLANG SERAI,0.472481,11


In [111]:
flat_park.to_csv('flat_park.csv', index=False)

# MRT

In [114]:
mrt_coord = pd.read_csv('mrt_lrt_data.csv')
mrt_coord = mrt_coord[['STN_NAME','Latitude','Longitude']]
mrt_coord.head(10)

,STN_NAME,Latitude,Longitude
0,Jurong East,1.333207,103.742308
1,Bukit Batok,1.349069,103.749596
2,Bukit Gombak,1.359043,103.751863
3,Choa Chu Kang,1.385417,103.744316
4,Yew Tee,1.397383,103.747523
5,Kranji,1.425302,103.762049
6,Marsiling,1.432579,103.774150
7,Woodlands,1.436984,103.786406
8,Admiralty,1.436984,103.786406
9,Sembawang,1.449133,103.820060


In [116]:
nearest_mrt = find_nearest(flat_coord, mrt_coord)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'mrt', 2: 'mrt_dist', 3: 'num_mrt_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head(10)

,flat,mrt,mrt_dist,num_mrt_2km
0,1 BEACH RD,Nicoll Highway,0.390848,16
1,1 BEDOK STH AVE 1,Bedok,0.52689,3
2,1 CANTONMENT RD,Outram Park,0.575026,9
3,1 CHAI CHEE RD,Bedok,0.939721,5
4,1 CHANGI VILLAGE RD,Changi Airport,3.420324,0
5,1 DELTA AVE,Tiong Bahru,0.636726,7
6,1 DOVER RD,one-north,0.563805,6
7,1 EUNOS CRES,Eunos,0.105065,6
8,1 EVERTON PK,Outram Park,0.575026,9
9,1 GEYLANG SERAI,Paya Lebar,0.578542,8


In [118]:
flat_mrt.to_csv('flat_mrt.csv', index=False)

In [122]:
flat_coord = pd.read_csv('HDB_coordinate_clean.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head(10)

,address,LATITUDE,LONGITUDE
0,1 BEACH RD,1.303671,103.864479
1,1 BEDOK STH AVE 1,1.320852,103.933721
2,1 CANTONMENT RD,1.275499,103.841398
3,1 CHAI CHEE RD,1.327969,103.922716
4,1 CHANGI VILLAGE RD,1.388547,103.987804
5,1 DELTA AVE,1.292075,103.828584
6,1 DOVER RD,1.302530,103.783272
7,1 EUNOS CRES,1.320658,103.902463
8,1 EVERTON PK,1.275499,103.841398
9,1 GEYLANG SERAI,1.316679,103.898103


In [128]:
# Merge all
flat_amenities = flat_school.merge(flat_hawker, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_park, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mall, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mrt, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_supermarket, on='flat', how='outer')

flat_amenities.head(10)

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,park_dist,num_park_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km
0,1 BEACH RD,Stamford Primary School,0.789938,2,Golden Mile Food Centre,0.110155,7,0.21358,14,Aperia,0.672618,18,Nicoll Highway,0.390848,16,0.125944,24
1,1 BEDOK STH AVE 1,Bedok Green Primary School,0.566936,7,Blk 16 Bedok South Road,0.198866,8,0.507003,22,Bedok Point,0.461313,5,Bedok,0.52689,3,0.115509,20
2,1 CANTONMENT RD,Cantonment Primary School,0.164246,5,Blk 6 Tanjong Pagar Plaza,0.232288,10,0.621771,14,100 AM,0.251648,10,Outram Park,0.575026,9,0.196809,18
3,1 CHAI CHEE RD,Damai Primary School,0.832069,8,Blk 538 Bedok North Street 3,0.498505,6,0.673814,16,Djitsun Mall,0.795355,4,Bedok,0.939721,5,0.399689,20
4,1 CHANGI VILLAGE RD,Pasir Ris Primary School,3.300565,0,Blks 2 & 3 Changi Village Road,0.090824,2,0.234404,7,Jewel Changi Airport,3.126753,0,Changi Airport,3.420324,0,3.325376,0
5,1 DELTA AVE,Alexandra Primary School,0.523839,7,Zion Riverside Food Centre,0.291435,9,0.386117,11,Great World City,0.390828,26,Tiong Bahru,0.636726,7,0.194051,31
6,1 DOVER RD,Fairfield Methodist School (Primary),0.330473,3,Blk 20 Ghim Moh Road,1.079151,5,0.661312,14,Rochester Mall,0.657988,4,one-north,0.563805,6,0.302828,16
7,1 EUNOS CRES,Eunos Primary School,0.464139,7,Blk 4A Eunos Crescent,0.210803,6,0.69627,14,Singapore Post Centre (Singpost Centre),0.87899,10,Eunos,0.105065,6,0.134991,23
8,1 EVERTON PK,Cantonment Primary School,0.164246,5,Blk 6 Tanjong Pagar Plaza,0.232288,10,0.621771,14,100 AM,0.251648,10,Outram Park,0.575026,9,0.196809,18
9,1 GEYLANG SERAI,Haig Girls' School,0.742367,8,Geylang Serai Market,0.014855,10,0.472481,11,Singapore Post Centre (Singpost Centre),0.443402,11,Paya Lebar,0.578542,8,0.015315,25


In [130]:
flat_amenities.to_csv('flat_amenities.csv', index=False)

In [132]:
flat_coord = pd.read_csv('HDB_coordinate.csv')
flat_coord = flat_coord[['Address','Latitude','Longitude']]

dist_dhoby = dist_from_location(flat_coord, (1.299308, 103.845285))
dist_dhoby = pd.DataFrame.from_dict(dist_dhoby).T
dist_dhoby = dist_dhoby.rename(columns={0: 'flat', 1: 'dist_dhoby'}).reset_index().drop(['index'], axis=1)
dist_dhoby.head(10)

,flat,dist_dhoby
0,1 BEACH RD,2.189896
1,1 BEDOK STH AVE 1,10.126306
2,1 CANTONMENT RD,2.667976
3,1 CHAI CHEE RD,9.18161
4,1 CHANGI VILLAGE RD,18.679811
5,1 DELTA AVE,2.023424
6,1 DOVER RD,6.910711
7,1 EUNOS CRES,6.787205
8,1 EVERTON PK,2.667976
9,1 GEYLANG SERAI,6.183984


In [134]:
flat_amenities = pd.read_csv('flat_amenities.csv')
flat_amenities = flat_amenities.merge(dist_dhoby, on='flat', how='outer')
flat_amenities.head(10)

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,park_dist,num_park_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_dhoby
0,1 BEACH RD,Stamford Primary School,0.789938,2,Golden Mile Food Centre,0.110155,7,0.213580,14,Aperia,0.672618,18,Nicoll Highway,0.390848,16,0.125944,24,2.189896
1,1 BEDOK STH AVE 1,Bedok Green Primary School,0.566936,7,Blk 16 Bedok South Road,0.198866,8,0.507003,22,Bedok Point,0.461313,5,Bedok,0.526890,3,0.115509,20,10.126306
2,1 CANTONMENT RD,Cantonment Primary School,0.164246,5,Blk 6 Tanjong Pagar Plaza,0.232288,10,0.621771,14,100 AM,0.251648,10,Outram Park,0.575026,9,0.196809,18,2.667976
3,1 CHAI CHEE RD,Damai Primary School,0.832069,8,Blk 538 Bedok North Street 3,0.498505,6,0.673814,16,Djitsun Mall,0.795355,4,Bedok,0.939721,5,0.399689,20,9.18161
4,1 CHANGI VILLAGE RD,Pasir Ris Primary School,3.300565,0,Blks 2 & 3 Changi Village Road,0.090824,2,0.234404,7,Jewel Changi Airport,3.126753,0,Changi Airport,3.420324,0,3.325376,0,18.679811
5,1 DELTA AVE,Alexandra Primary School,0.523839,7,Zion Riverside Food Centre,0.291435,9,0.386117,11,Great World City,0.390828,26,Tiong Bahru,0.636726,7,0.194051,31,2.023424
6,1 DOVER RD,Fairfield Methodist School (Primary),0.330473,3,Blk 20 Ghim Moh Road,1.079151,5,0.661312,14,Rochester Mall,0.657988,4,one-north,0.563805,6,0.302828,16,6.910711
7,1 EUNOS CRES,Eunos Primary School,0.464139,7,Blk 4A Eunos Crescent,0.210803,6,0.696270,14,Singapore Post Centre (Singpost Centre),0.878990,10,Eunos,0.105065,6,0.134991,23,6.787205
8,1 EVERTON PK,Cantonment Primary School,0.164246,5,Blk 6 Tanjong Pagar Plaza,0.232288,10,0.621771,14,100 AM,0.251648,10,Outram Park,0.575026,9,0.196809,18,2.667976
9,1 GEYLANG SERAI,Haig Girls' School,0.742367,8,Geylang Serai Market,0.014855,10,0.472481,11,Singapore Post Centre (Singpost Centre),0.443402,11,Paya Lebar,0.578542,8,0.015315,25,6.183984


In [136]:
flat_amenities.to_csv('flat_amenities.csv', index=False)